In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import csv
from contextlib import ExitStack
import progressbar as pbar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
male_cases = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=maleCases&format=csv'
female_cases = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=femaleCases&format=csv'

male_cases = pd.read_csv(male_cases)
female_cases = pd.read_csv(female_cases)

In [3]:
translator = {'0_to_4': '00-04', '5_to_9': '05-09',
              '10_to_14': '10-14', '15_to_19': '15-19', 
              '20_to_24': '20-24', '25_to_29': '25-29', 
              '30_to_34': '30-34', '35_to_39': '35-39', 
              '40_to_44': '40-44', '45_to_49': '45-49', 
              '50_to_54': '50-54', '55_to_59': '55-59', 
              '60_to_64': '60-64', '65_to_69': '65-69', 
              '70_to_74': '70-74', '75_to_79': '75-79', 
              '80_to_84': '80-84', '85_to_89': '85-89',}

In [4]:
today = datetime.date.today().strftime("%d-%m-%Y")
# today = '15-02-2022'
data_dir_path = f'C:\\Users\\macie\\Desktop\\Programowanie\\Python\\dark-figure\\data\\World\\{today}'
os.path.abspath(data_dir_path)
# if not os.path.exists(f'{data_dir_path}'):
#     os.mkdir(f'{data_dir_path}')

'C:\\Users\\macie\\Desktop\\Programowanie\\Python\\dark-figure\\data\\World\\03-03-2022'

In [5]:
# male_cases = male_cases.drop(columns=['areaCode', 'areaType', 'rate'])
cases = male_cases.append(female_cases)
cases = cases.groupby(['date', 'age']).sum().reset_index().drop(columns=['rate'])
cases['date'] = cases['date'].apply(pd.to_datetime)
cases = cases.groupby(['date', 'age']).sum().reset_index()
cases = cases.replace(translator).rename(columns={'age': 'Age group'})


fixed_deaths = pd.DataFrame(columns=['date', 'location', 'location_name', 'Age group', 'value'])
for age_group in cases['Age group'].unique():
    deaths_age_group = cases[cases['Age group'] == age_group].groupby('date')['value'].sum().reset_index()

    fixed_deaths_age_group = deaths_age_group['value']
    
    fixed_deaths_age_group[fixed_deaths_age_group <= 0]=np.nan
    fixed_deaths_age_group = fixed_deaths_age_group.dropna()
    fixed_deaths_age_group = np.pad(fixed_deaths_age_group, (len(deaths_age_group) - len(fixed_deaths_age_group), 0), 'constant', constant_values=(0,0))
    z = fixed_deaths_age_group[1:] - fixed_deaths_age_group[:-1]
    fixed_deaths_age_group[:-1][z<0]=np.nan
    fixed_deaths_age_group = pd.Series(fixed_deaths_age_group)
    fixed_deaths_age_group.index = deaths_age_group['date']
    
    fixed_deaths_age_group = fixed_deaths_age_group.resample('D').mean()
    fixed_deaths_age_group = fixed_deaths_age_group.interpolate()
        
    removed_negative_deaths = np.insert(np.maximum(0, (fixed_deaths_age_group.values[1:] - fixed_deaths_age_group.values[:-1])), 0, 0)
    fixed_deaths_age_group = pd.DataFrame(fixed_deaths_age_group)
    fixed_deaths_age_group['value'] = removed_negative_deaths
    fixed_deaths = fixed_deaths.append(fixed_deaths_age_group.reset_index())
    fixed_deaths['location'] = fixed_deaths['location'].fillna('United Kingdom')
    fixed_deaths['location_name'] = fixed_deaths['location_name'].fillna('UK')
    fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(age_group)

fixed_deaths = fixed_deaths.append(fixed_deaths[fixed_deaths['Age group'].isin(['85-89', '90+'])].groupby(['date', 'location', 'location_name']).sum().reset_index())
fixed_deaths = fixed_deaths[~fixed_deaths['Age group'].isin(['85-89', '90+'])]
fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(f'85+')

fixed_deaths = fixed_deaths.sort_values('date')
fixed_deaths.to_csv(f'{data_dir_path}\\United Kingdom\\covid_cases.csv')
fixed_deaths['Age group'].unique()

array(['00-04', '15-19', '80-84', '20-24', '75-79', '25-29', '70-74',
       '85+', '30-34', '65-69', '40-44', '60-64', '45-49', '05-09',
       '50-54', '35-39', '10-14', '55-59'], dtype=object)

In [6]:
female_deaths_link = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=femaleDeaths28Days&format=csv&release='
male_deaths_link = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&metric=maleDeaths28Days&format=csv&release='
data = pd.DataFrame()
for date in pd.date_range(start='3/2/2020', end='01/3/2022'):
    try:
        new_date = pd.read_csv(female_deaths_link + date.strftime("%Y-%m-%d"))
        data = data.append(new_date)
        new_date = pd.read_csv(male_deaths_link + date.strftime("%Y-%m-%d"))
        data = data.append(new_date)
    except BaseException as err:
        pass
deaths = data.groupby(['date', 'age']).sum().reset_index().sort_values('date')
deaths = deaths.drop(columns=['rate'])
deaths['date'] = deaths['date'].apply(pd.to_datetime)
deaths = deaths.replace(translator).rename(columns={'age': 'Age group'})


fixed_deaths = pd.DataFrame(columns=['date', 'location', 'location_name', 'Age group', 'value'])
for age_group in deaths['Age group'].unique():
    deaths_age_group = deaths[deaths['Age group'] == age_group].groupby('date')['value'].sum().reset_index()

    fixed_deaths_age_group = deaths_age_group['value']
    
    fixed_deaths_age_group[fixed_deaths_age_group <= 0]=np.nan
    fixed_deaths_age_group = fixed_deaths_age_group.dropna()
    fixed_deaths_age_group = np.pad(fixed_deaths_age_group, (len(deaths_age_group) - len(fixed_deaths_age_group), 0), 'constant', constant_values=(0,0))
    z = fixed_deaths_age_group[1:] - fixed_deaths_age_group[:-1]
    # fixed_deaths_age_group[:-1][z<0]=np.nan
    fixed_deaths_age_group = pd.Series(fixed_deaths_age_group)
    fixed_deaths_age_group.index = deaths_age_group['date']
    
    fixed_deaths_age_group = fixed_deaths_age_group.resample('D').mean()
    fixed_deaths_age_group = fixed_deaths_age_group.interpolate()
        
    removed_negative_deaths = np.insert(np.maximum(0, (fixed_deaths_age_group.values[1:] - fixed_deaths_age_group.values[:-1])), 0, 0)
    fixed_deaths_age_group = pd.DataFrame(fixed_deaths_age_group)
    fixed_deaths_age_group['value'] = removed_negative_deaths
    fixed_deaths = fixed_deaths.append(fixed_deaths_age_group.reset_index())
    fixed_deaths['location'] = fixed_deaths['location'].fillna('United Kingdom')
    fixed_deaths['location_name'] = fixed_deaths['location_name'].fillna('UK')
    fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(age_group)

fixed_deaths = fixed_deaths.append(fixed_deaths[fixed_deaths['Age group'].isin(['85-89', '90+'])].groupby(['date', 'location', 'location_name']).sum().reset_index())
fixed_deaths = fixed_deaths[~fixed_deaths['Age group'].isin(['85-89', '90+'])]
fixed_deaths['Age group'] = fixed_deaths['Age group'].fillna(f'85+')
fixed_deaths = fixed_deaths.sort_values('date')
fixed_deaths.to_csv(f'{data_dir_path}/United Kingdom/covid_deaths.csv')
fixed_deaths

,date,location,location_name,Age group,value,0
0,2021-07-05,United Kingdom,UK,00-04,0.000000,10.0
0,2021-07-05,United Kingdom,UK,70-74,0.000000,11010.0
0,2021-07-05,United Kingdom,UK,80-84,0.000000,20368.0
0,2021-07-05,United Kingdom,UK,60-64,0.000000,3884.0
0,2021-07-05,United Kingdom,UK,05-09,0.000000,6.0
...,...,...,...,...,...,...
181,2022-01-02,United Kingdom,UK,25-29,0.666667,161.0
181,2022-01-02,United Kingdom,UK,10-14,0.000000,20.0
181,2022-01-02,United Kingdom,UK,35-39,1.333333,532.0
181,2022-01-02,United Kingdom,UK,65-69,6.333333,9664.0
